In [263]:
import pandas as pd
import numpy as np
import plotly.express as px

In [264]:
# Read data
df = pd.read_csv('Data/archive/Melbourne_Housing_Data_Cleaned.csv')

In [265]:
df.head()

,Suburb,Address,Rooms,Type,Price,Date,Distance,Postcode,Bathroom,Car,LandArea,Council,Latitude,Longitude,Region,State
0,Abbotsford,85 Turner St,2,h,1480000.0,3/12/2016,2.5,3067,1,1,202.0,Yarra City Council,-37.7996,144.9984,Northern Metropolitan,VIC
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,4/02/2016,2.5,3067,1,0,156.0,Yarra City Council,-37.8079,144.9934,Northern Metropolitan,VIC
2,Abbotsford,5 Charles St,3,h,1465000.0,4/03/2017,2.5,3067,2,0,134.0,Yarra City Council,-37.8093,144.9944,Northern Metropolitan,VIC
3,Abbotsford,5 Charles St,3,h,1465000.0,4/03/2017,2.5,3067,2,0,134.0,Yarra City Council,-37.8093,144.9944,Northern Metropolitan,VIC
4,Abbotsford,5 Charles St,3,h,1465000.0,4/03/2017,2.5,3067,2,0,134.0,Yarra City Council,-37.8093,144.9944,Northern Metropolitan,VIC


In [266]:
# Input (instead of asking user to input we can have a dropdown with available suburb names to choose from to avoid nulls)
suburb = input()

Caulfield North


In [267]:
# Filter data for provided suburb
df_filtered = df[df['Suburb'] == suburb]

In [269]:
df_filtered.head()

,Suburb,Address,Rooms,Type,Price,Date,Distance,Postcode,Bathroom,Car,LandArea,Council,Latitude,Longitude,Region,State
1973,Caulfield North,14 Arthur St,3,h,1365000.0,3/09/2016,8.1,3161,1,0,513.0,Glen Eira City Council,-37.8678,145.0270,Southern Metropolitan,VIC
1974,Caulfield North,14 Arthur St,3,h,1365000.0,3/09/2016,8.1,3161,1,0,513.0,Glen Eira City Council,-37.8678,145.0270,Southern Metropolitan,VIC
1975,Caulfield North,14 Arthur St,3,h,1365000.0,3/09/2016,8.1,3161,1,0,513.0,Glen Eira City Council,-37.8678,145.0270,Southern Metropolitan,VIC
1976,Caulfield North,452 Dandenong Rd,3,h,1750000.0,3/09/2016,8.1,3161,2,2,692.0,Glen Eira City Council,-37.8629,145.0190,Southern Metropolitan,VIC
1977,Caulfield North,12/378 Dandenong Rd,2,u,580000.0,4/03/2017,8.1,3161,1,1,934.0,Glen Eira City Council,-37.8607,145.0116,Southern Metropolitan,VIC


In [270]:
# Convert 'Date' column to datetime format
df_filtered['Date'] = pd.to_datetime(df_filtered['Date'], format='%d/%m/%Y')

In [271]:
# Extract month and year from 'Date' column
df_filtered['Year'] = df_filtered['Date'].dt.year
df_filtered['Month'] = df_filtered['Date'].dt.month

In [272]:
# Group by month number and year to count the number of sales
sales_per_month = df_filtered.groupby(['Year', 'Month']).size().reset_index(name='Number of Sales')

In [273]:
sales_per_month

,Year,Month,Number of Sales
0,2016,5,3
1,2016,8,2
2,2016,9,6
3,2016,10,3
4,2016,11,2
5,2016,12,2
6,2017,2,4
7,2017,3,2
8,2017,4,3
9,2017,5,5


In [274]:
# Map month numbers to month names
month_map = {
    1: 'January', 2: 'February', 3: 'March', 4: 'April', 5: 'May', 6: 'June',
    7: 'July', 8: 'August', 9: 'September', 10: 'October', 11: 'November', 12: 'December'
}
sales_per_month['Month_Name'] = sales_per_month['Month'].map(month_map)

In [275]:
# Create a bar chart with Plotly
fig = px.line(sales_per_month, x='Month_Name', y='Number of Sales', color='Year', 
             title='Number of Sales per Month', 
             labels={'Month_Name': 'Month', 'Number of Sales': 'Number of Sales'},
             category_orders={"Month_Name": list(month_map.values())})

In [276]:
# Set y-axis to display only whole numbers and adjust margin for x-axis
max_sales = sales_per_month['Number of Sales'].max()
fig.update_layout(yaxis=dict(tickvals=list(range(0, max_sales+1))))
# Show plot
fig.show()

In [277]:
# Group by year to get total sales for each year
total_sales_per_year = df_filtered.groupby('Year').size().reset_index(name='Total Sales')

In [278]:
total_sales_per_year

,Year,Total Sales
0,2016,18
1,2017,20


In [279]:
# Calculate the percentage change from the previous year
total_sales_per_year['Percentage Change'] = total_sales_per_year['Total Sales'].pct_change() * 100


In [280]:
# Create a new column for the year interval representation
total_sales_per_year['Year Interval'] = (total_sales_per_year['Year'] - 1).astype(str) + '-' + total_sales_per_year['Year'].astype(str)


In [281]:
# Drop the first row since we don't have percentage change data for the first year
total_sales_per_year = total_sales_per_year.dropna()


In [282]:
total_sales_per_year

,Year,Total Sales,Percentage Change,Year Interval
1,2017,20,11.111111,2016-2017
